In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from utils.reparam_module import ReparamModule
import numpy as np

In [14]:
dataset_name = 'beauty'
full_dataset_name = 'amazon-beauty'
min_frequency_dict = {
    'toys': 2,
    'sport': 2,
    'beauty': 2,
    'yelp': 2,
}
max_span_dict = {
    'toys': 10,
    'sport': 10,
    'beauty': 5,
    'yelp': 10,
}
min_frequency = min_frequency_dict[dataset_name]
max_span = max_span_dict[dataset_name]
data = torch.load(f'./pattern-{dataset_name}-50.pth')
print(len(data))

22363


In [15]:
# Example to show how to find frequent sequential patterns
# from a given sequence database subject to constraints
from sequential.seq2pat import Seq2Pat, Attribute

# Seq2Pat over 3 sequences
seq2pat = Seq2Pat(sequences=data, n_jobs=2, max_span=max_span)
# Patterns that occur at least twice (A-D)
patterns = seq2pat.get_patterns(min_frequency=min_frequency)
patterns_value = [_[:-1] for _ in patterns]
patterns_count = [_[-1] for _ in patterns]
print(len(patterns))

40664


In [16]:
original_train = torch.load(f'./dataset/{full_dataset_name}-50/{dataset_name}/train_ori.pth')
len(original_train)

22363

In [17]:
seq_list_ori = [_[1][:_[3]] + [_[2][_[3] - 1]] for _ in original_train]

In [18]:
def is_sublist(sublst, lst):
    for element in sublst:
        try:
            ind = lst.index(element)
        except ValueError:
            return False
        lst = lst[ind+1:]
    return True

In [19]:
data_generation_pair = []
total_cnt = 0
from tqdm import tqdm
from random import shuffle
for seq_ori in tqdm(seq_list_ori):
    shuffle(patterns_value)
    cnt = 0
    for pattern in patterns_value:
        if is_sublist(pattern, seq_ori):
            data_generation_pair.append([seq_ori, pattern])
            cnt += 1
        if cnt == 10:
            break
print(len(data_generation_pair))

100%|██████████| 22363/22363 [18:28<00:00, 20.18it/s]

44471


In [20]:
torch.save(data_generation_pair, f'./{dataset_name}-pair-2-5.pth')